# Homework 3. Segmentation.

Снова детектируем края! Вместо  Сanny edge detector, обучите unet-like сетку сегментировать края клеток.

В этом задании вам не будет предоставлено практически никакого кода, только входные данные и метрика - interesection-over-union (IoU).

Максимальное количество баллов, которое можно будет получить за домашку, - **80**. Дедлайн - 12 июля 23:59.

Вам нужно обучить сетку предсказывать маску границ клеток.

Используйте все, что вы знаете на данный момент:
* Любую архитектуру для семантической сегментации (encoder-decoder или на основе dilated convolusion). Рекомендую использовать UNet.

* data augmentation (она точно понадобится поскольку трейн сет состоит только из 41 изображения)

* fine-tuning 

Запрещается делать только одно: обучать сетку на тестовом датасете

Засабмитить нужно будет заполненный ноутбук с кодом, с посчитанным значением iou на валидационном датасете, и нарисовать получившиеся изображения для тестового датасета (одноканальное, бинарное изображение, 1 - есть граница, 0 - нету границы).

Хорошая сетка должна сегментировать изображения с val iou >= 0.28. Это не строгий критерий для получения полного балла, но попытайтесь получить схожие значения метрики.

Практические советы:
* На семинаре мы уже реализовали Unet, можете использовать код из семинара. 
* Скорей всего вы будете создавать собственный класс-датасет как наследник класса `torch.data.utils.Dataset`. Посмотреть, как это  сделать, можно [тут](https://discuss.pytorch.org/t/dataloader-for-semantic-segmentation/48290).
* В датасете большой имбаланс классов, поэтому сетка будет выдавать больше "нулевой" (не-граница) класс. Чтобы справиться с этой проблемой, следует либо изменить порог при назначении класов (вместо 0.5 выбрать порог поменьше), либо задать веса для классов в лосс-функции (параметр `weights` в `nn.CrossentropyLoss`). Для единичного класса бОльший вес.
* У вас маленький датасет, поэтому нужно активно использовать аугментации. Обратите внимание, что если вы используете какие-либо геометрические аугментации (повороты, кропы, флипы, ...) для изображения, ту же самую аугментацию надо выполнять и на маску. Для этого есть два пути: либо кастомизировать пайторчевский датасет (см. [тут](https://discuss.pytorch.org/t/torchvision-transfors-how-to-perform-identical-transform-on-both-image-and-target/10606/5)), либо использовать Albumentations.
* По желанию, можно добавить различные пост-процессинг трюки (морфологические операции, локальный трешхолдинг), но лучше посвятить свое время улучшению работы сетки.

Удачи!



In [0]:
### Download the dataset ###
!wget https://www.dropbox.com/s/jy34yowcf85ydba/data.zip?dl=0 -O data.zip
!unzip -q data.zip

In [0]:
### Visualize the data ###
import matplotlib.pyplot as plt
import numpy as np
import skimage
from skimage import io
%matplotlib inline



# Human HT29 colon-cancer cells
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
im = skimage.img_as_ubyte(io.imread('BBBC018_v1_images-fixed/train/00735-actin.DIB.bmp'))
plt.imshow(im)
plt.subplot(1,2,2)
mask = skimage.img_as_ubyte(io.imread('BBBC018_v1_outlines/train/00735-cells.png'))
plt.imshow(mask, 'gray')

In [0]:
### Target metric ###
def calc_iou(prediction, ground_truth):
    n_images = len(prediction)
    intersection, union = 0, 0
    for i in range(n_images):
        intersection += np.logical_and(prediction[i] > 0, ground_truth[i] > 0).astype(np.float32).sum() 
        union += np.logical_or(prediction[i] > 0, ground_truth[i] > 0).astype(np.float32).sum()
    return float(intersection) / union

In [0]:
# YOUR CODE HERE